# CCNMF analysis of simulated paired scRNA and scDNA data

In [2]:
# Load necessary packages
library(CCNMF)
library(Seurat)
library(NMF)
library(pheatmap)
library(ggplot2)
library(cowplot)
library(ggplotify)
library(Rtsne)

In [3]:
# Load Data
CNVmatrix <- as.matrix(read.csv('~/CCNMF/testdata/DNA.csv'))
RNAmatrix <- as.matrix(read.csv('~/CCNMF/testdata/RNA.csv'))

In [4]:
# Preprocessing scRNA-seq data by Seurat
rownames(RNAmatrix) <- paste0('gene', 1:dim(RNAmatrix)[1])
colnames(RNAmatrix) <- paste0('cell', 1:dim(RNAmatrix)[2])
RNAobject <- run_Seurat_RNA(RNAmatrix)

Centering and scaling data matrix

PC_ 1 
Positive:  gene247, gene1221, gene1256, gene729, gene346, gene681, gene1894, gene1987, gene738, gene861 
	   gene1411, gene1557, gene1113, gene110, gene776, gene1858, gene911, gene505, gene467, gene181 
	   gene205, gene711, gene273, gene933, gene477, gene593, gene835, gene1958, gene853, gene1801 
Negative:  gene1218, gene1064, gene914, gene218, gene955, gene1594, gene1111, gene540, gene757, gene1744 
	   gene169, gene576, gene935, gene732, gene626, gene1712, gene336, gene1580, gene580, gene1947 
	   gene768, gene804, gene1166, gene837, gene219, gene1920, gene1585, gene115, gene1873, gene162 
PC_ 2 
Positive:  gene246, gene260, gene1958, gene1801, gene470, gene737, gene776, gene537, gene205, gene911 
	   gene754, gene819, gene711, gene1692, gene1471, gene681, gene685, gene1225, gene1337, gene1564 
	   gene1640, gene1734, gene1639, gene1128, gene1886, gene1408, gene1545, gene589, gene1951, gene945 
Negative:  gene247, gene1256, gene1221, gene198

Modularity Optimizer version 1.3.0 by Ludo Waltman and Nees Jan van Eck

Number of nodes: 1000
Number of edges: 37717

Running Louvain algorithm...
Maximum modularity in 10 random starts: 0.9275
Number of communities: 3
Elapsed time: 0 seconds


In [5]:
# Then we use a sigmoid fucnction transform the negative elements in scaled RNA matrix to non-negative 
# since the CCNMF needs non-negative matrices as input.
sigmoid <- function(x){1/(1+ exp(-x))}
RNAmatrix1 <- RNAobject@assays$RNA@scale.data[RNAobject@assays$RNA@meta.features$vst.variable, ]
replaceindex <- which(RNAmatrix1 < 0 )
RNAmatrix2 <- RNAmatrix1
RNAmatrix2[replaceindex] <- sigmoid(RNAmatrix1)[replaceindex]

In [6]:
# Preprocessing scDNA-seq data 
rownames(CNVmatrix) <- paste0('gene', 1:dim(CNVmatrix)[1])
colnames(CNVmatrix) <- paste0('cell', 1:dim(CNVmatrix)[2])
CNVmatrix1 <- CNVmatrix[RNAobject@assays$RNA@meta.features$vst.variable, ]

In [7]:
# Set the coupledMatrix as identify matrix 
# CoupledMatrix <- as(as.matrix(diag(dim(RNAmatrix1)[1]), 'dgCMatrix')
CoupledMatrix <- as.matrix(diag(dim(RNAmatrix1)[1]))

[1] 2000 2000

In [9]:
# Run CCNMF
ResultsCCNMF <- run_CCNMF(ncluster = 3, CNVmatrix1, RNAmatrix2, CoupledMatrix, lambda1=1, lambda2=1, mu=1)
length(ResultsCCNMF)

[1] "Initializing NMF for CNV matrirx..."
[1] "Initializing NMF for scRNA matrirx..."
[1] "Initializing the parameters lambda1, lambda2 and mu..."
[1] "Start Coupled NMF"
[1] "Iterating coupledNMF..."
[1] "eq1 157817"
[1] "eq2 501441"
[1] "eq3 8496"
[1] "eq4 22315"
[1] "eq5 659257"
[1] "Run time:78.598seconds"


[1] 7

In [10]:
# Find the differential genes between clusters based on scRNA data
DElist <- DiffExp(RNAmatrix2, ResultsCCNMF[[6]])

[1] 3

In [ ]:
# Output the heatmap and tsne figures of CCNMF, the main figure's pdf can be saved in current path.
PlotMainResult(CNVmatrix1, RNAmatrix2, ResultsCCNMF, DElist)

![](simulation_figure.png)

The result figure can be saved in the current path.